In [1]:
import intake
import satstac
import yaml
import inspect
import intake_xarray
import intake_stac
intake_stac.__version__

'0.3.0.post58'

# Load an Intake Catalog, Grab a Single Asset

In [2]:
url = 'https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/catalog.json'
stac_cat = intake.open_stac_catalog(url)

l1 = stac_cat
l2 = stac_cat['extensions-collection']
l3 = stac_cat['extensions-collection']['proj-example']

print(list(l1))
print(list(l2))
print(list(l3))

['extensions-collection', 'sentinel-2', 'CS3-20160503_132131_08']
['proj-example']
['B1', 'B8']


In [3]:
b1 = stac_cat['extensions-collection']['proj-example']['B1']
b8 = stac_cat['extensions-collection']['proj-example']['B8']

print(type(b1))
print(b1)

<class 'intake_xarray.raster.RasterIOSource'>
sources:
  stac_asset:
    args:
      chunks: {}
      urlpath: https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF
    description: Band 1 (coastal)
    driver: intake_xarray.raster.RasterIOSource
    metadata:
      catalog_dir: ''
      eo:bands:
      - center_wavelength: 0.44
        common_name: coastal
        full_width_half_max: 0.02
        name: B1
      href: https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF
      plots:
        geotiff:
          cmap: viridis
          data_aspect: 1
          dynamic: true
          frame_width: 500
          kind: image
          rasterize: true
          x: x
          y: y
      title: Band 1 (coastal)
      type: image/tiff; application=geotiff



# Serialize as a .YAML file

[Intake _yaml code](https://github.com/intake/intake/blob/471488801bfa338dbfd91f9851b27580f4f3c3bb/intake/source/base.py#L242-L265)

``` python
def _yaml(self):
    import inspect
    kwargs = self._captured_init_kwargs.copy()
    meta = kwargs.pop('metadata', self.metadata) or {}
    kwargs.update(dict(zip(inspect.signature(self.__init__).parameters,
                       self._captured_init_args)))
    data = {
        'sources':
            {self.name: {
                'driver': self.classname,
                'description': self.description or "",
                'metadata': meta,
                'args': kwargs
            }}}
    return data

def yaml(self):
    """Return YAML representation of this data-source
    The output may be roughly appropriate for inclusion in a YAML
    catalog. This is a best-effort implementation
    """
    data = self._yaml()
    return yaml.dump(data, default_flow_style=False)
```

In [4]:
b1._yaml() # For datasources the existing yaml implimentation works.

{'sources': {'stac_asset': {'driver': 'intake_xarray.raster.RasterIOSource',
   'description': 'Band 1 (coastal)',
   'metadata': {'href': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF',
    'type': 'image/tiff; application=geotiff',
    'title': 'Band 1 (coastal)',
    'eo:bands': [{'name': 'B1',
      'common_name': 'coastal',
      'center_wavelength': 0.44,
      'full_width_half_max': 0.02}],
    'plots': {'geotiff': {'kind': 'image',
      'x': 'x',
      'y': 'y',
      'frame_width': 500,
      'data_aspect': 1,
      'rasterize': True,
      'dynamic': True,
      'cmap': 'viridis'}},
    'catalog_dir': ''},
   'args': {'urlpath': 'https://landsat-pds.s3.amazonaws.com/c1/L8/107/018/LC08_L1TP_107018_20181001_20181001_01_RT/LC08_L1TP_107018_20181001_20181001_01_RT_B1.TIF',
    'chunks': {}}}}}

In [5]:
## Go up one level to `stac_cat['extensions-collection']['proj-example']` 
#print(l3.yaml()) # Contains the problematic typing `&id001 !!python/object:pystac.item.Item` throughout
#print(l3.serialize()) # Contains the problematic typing `&id001 !!python/object:pystac.item.Item` throughout

kwargs = l3._captured_init_kwargs.copy()
meta = kwargs.pop('metadata', l3.metadata) or {}
#print(kwargs)
kwargs.pop('stac_obj') # Remove this arg that maps to the problematic typing and see if works
kwargs.update(dict(zip(inspect.signature(l3.__init__).parameters,
                l3._captured_init_args)))

l3data = {
    'driver': l3.classname,
    'description': l3.description,
    'metadata': l3.metadata,
    'args': kwargs
        }

l3dict = {
    'sources':
        {l3.name: l3data}
        }

with open('l3test.yaml', 'w') as f:
    f.write(yaml.dump(l3dict, sort_keys=False)) #Can be read in!

In [6]:
## Now include the sub catalogs b1 and b8

data = {'metadata':{'version':1}, 'sources':{}}
data['sources'][l3.name]=l3data

for key, source in l3.items():
    data['sources'][l3.name][key] = source._yaml()['sources']

# Should B1 and B8 be on the same level as proj-example? 
# Maybe remove the key 'stac_asset' but keep all the content?

In [7]:
print(yaml.dump(data, sort_keys=False))

metadata:
  version: 1
sources:
  proj-example:
    driver: intake_stac.catalog.StacItem
    description: ''
    metadata:
      datetime: 2018-10-01 01:08:32.033000+00:00
      proj:epsg: 32614
      proj:wkt2: PROJCS["WGS 84 / UTM zone 14N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS
        84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],AUTHORITY["EPSG","32614"],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
      proj:projjson:
        $schema: https://proj.org/schemas/v0.2/projjson.schema.json
        type: ProjectedCRS
        name: WGS 84 / UTM zone 14N
        base_crs:
          

In [8]:
with open('test.yaml', 'w') as f:
    f.write(yaml.dump(data, sort_keys=False))

# Full Circle: Load Intake Catalogue from .YAML

In [16]:
catalog2 = intake.open_catalog('test.yaml')
list(catalog2)

['proj-example']

In [ ]:
catalog2['proj-example'] #Type Error missing `stac_obj` - the key that I popped, so perhaps I did something taboo